<a href="https://colab.research.google.com/github/elianderlohr/muse-dlf/blob/main/notebooks/explainablity/muse-dlf-explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SLMuSE-DLF Explainability

Plot the explainability of the SLMuSE-DLF model. By using the dictionary learning approach it is (1.) possible to extract how different words in a certaim semantic role predict the presence of a document level frame and (2.) identify how the FrameAxis constallations are predicting the document level frames.

In [ ]:
# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Git/muse-dlf

assets	data  labels_data.npz  notebooks  README.md  research-notebooks  run  src  tests  ToDo.md


In [ ]:
# !pip install allennlp allennlp-models

In [ ]:
!pip spacy torch==2.3.1

ERROR: unknown command "spacy"


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install wandb==0.17.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Git/muse-dlf/src')

In [ ]:
from preprocessing.pre_processor import PreProcessor
from preprocessing.datasets.article_dataset import custom_collate_fn
from model.muse_dlf.muse import MUSEDLF

# import tokenizer for roberta fast
from transformers import RobertaTokenizerFast
import wandb
import inspect
import torch
import spacy
import pickle
from pathlib import Path
from torch.utils.data import DataLoader

In [ ]:
wandb.require("core")

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
base_path = "drive/MyDrive/Git/"

## Setup wandb

In [ ]:
run = wandb.init(project="muse-dlf", job_type="inference")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Clean

In [ ]:
def clean_gpu_memory():
    # Clear cache
    torch.cuda.empty_cache()
    # Reset peak memory stats
    torch.cuda.reset_peak_memory_stats()
    # Perform garbage collection
    import gc
    gc.collect()

# clean_gpu_memory()

## Load MuSE-DLF

In [ ]:
model_artifact = run.use_artifact('elianderlohr/muse-dlf/crashing_backpack_2861_model:v8', type='model')
model_dir = model_artifact.download()

wandb: Downloading large artifact crashing_backpack_2861_model:v8, 5168.18MB. 2 files... 
Done. 0:0:0.1


## Load Roberta Model

In [ ]:
roberta_artifact = run.use_artifact('elianderlohr-org/wandb-registry-model/semeval-roberta-finetune:v1', type='model')
roberta_dir = roberta_artifact.download()

wandb: Downloading large artifact semeval-roberta-finetune:v1, 1427.27MB. 7 files... 
Done. 0:0:0.1


## Load Dataset

In [ ]:
dataset_artifact = run.use_artifact('elianderlohr-org/wandb-registry-dataset/muse-dlf:v1', type='dataset')
dataset_dir = dataset_artifact.download()

CommError: artifact 'muse-dlf:v1' not found in 'elianderlohr-org/wandb-registry-dataset'

## Load Unlabeled Dataset

In [ ]:
dataset_unlabeled_artifact = run.use_artifact('elianderlohr-org/wandb-registry-dataset/muse-dlf-unlabeled:v2', type='dataset')
dataset_unlabeled_dir = dataset_unlabeled_artifact.download()

In [ ]:
!ls /content/artifacts/semeval-unlabeled-dataset:v2

ls: cannot access '/content/artifacts/semeval-unlabeled-dataset:v2': No such file or directory


## Load Config

In [ ]:
# Access the run that created the artifactelianderlohr/slmuse-dlf/qa9dh6px
run_id = 'elianderlohr/muse-dlf/dmkklvop'  # Replace with your run ID if known, otherwise see below for how to get it
run_ref = wandb.Api().run(run_id)

In [ ]:
# Access the configuration
config = run_ref.config

In [ ]:
# Get the parameters of the SLMUSEDLF class constructor
params = inspect.signature(MUSEDLF.__init__).parameters

# Extract the relevant parameters from the config dictionary
model_params = {key: config[key] for key in params if key in config}

In [ ]:
model_params["bert_model_name"] = "roberta-base"
model_params["bert_model_name_or_path"] = roberta_dir
# model_params["_debug"] = True

## Load Model

In [ ]:
model = MUSEDLF(**model_params)

Some weights of RobertaModel were not initialized from the model checkpoint at /content/artifacts/roberta-base-finetune-checkpoint-16482:v0 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.set_log_level("INFO")

In [ ]:
# Assuming you load the state_dict as follows
state_dict = torch.load(f"{model_dir}/pytorch_model.bin", map_location="cpu")

In [ ]:
model.load_state_dict(state_dict)

<All keys matched successfully>

## Create Dataset

In [ ]:
!ls drive/MyDrive/Git/muse-dlf/data/axis

732_semaxis_axes.tsv  mft_experiment.json  mft_raw.csv
custom.tsv	      mft_filtered.csv	   plutchik_wheel_of_emotions.tsv
frames.json	      mft.json		   wordnet_antonyms.tsv


In [ ]:
class_column_names = "Capacity and Resources;Crime and Punishment;Cultural Identity;Economic;External Regulation and Reputation;Fairness and Equality;Health and Safety;Legality, Constitutionality, Jurisdiction;Morality;Other;Policy Prescription and Evaluation;Political;Public Sentiment;Quality of Life;Security and Defense".split(";")

### Create Full Dataset

In [ ]:
# Define paths to the dataset files within the downloaded directory
train_artifact_filepath = Path(dataset_dir) / 'train_dataset_artifact.pkl'
test_artifact_filepath = Path(dataset_dir) / 'test_dataset_artifact.pkl'

# Load the datasets from the artifact files
with train_artifact_filepath.open("rb") as f:
    loaded_train_dataset = pickle.load(f)

with test_artifact_filepath.open("rb") as f:
    loaded_test_dataset = pickle.load(f)

In [ ]:
# create dataloaders
train_dataloader = DataLoader(
    loaded_train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=custom_collate_fn,
    drop_last=True,
    pin_memory=True,
    num_workers=1,
)

test_dataloader = DataLoader(
    loaded_test_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=custom_collate_fn,
    drop_last=True,
    pin_memory=True,
    num_workers=1,
)

### Ceate Unlabeled Dataset

In [ ]:
unlabeled_artifact_filepath = Path(dataset_unlabeled_dir) / 'dataset_artifact.pkl'

# Load the datasets from the artifact files
with unlabeled_artifact_filepath.open("rb") as f:
    loaded_unlabeled_dataset = pickle.load(f)

# create dataloaders
unlabeled_dataloader = DataLoader(
    loaded_unlabeled_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    collate_fn=custom_collate_fn,
    drop_last=False,
    pin_memory=True,
    num_workers=1,
)

In [ ]:
len(loaded_unlabeled_dataset), len(unlabeled_dataloader)

(54, 7)

### Create Example Dataset

In [ ]:
preprocessor = PreProcessor(
    tokenizer=tokenizer,
    batch_size=config["batch_size"],
    max_sentences_per_article=config["num_sentences"],
    max_sentence_length=config["max_sentence_length"],
    max_args_per_sentence=config["max_args_per_sentence"],
    max_arg_length=config["max_arg_length"],
    frameaxis_dim=config["frameaxis_dim"],
    bert_model_name="roberta-base",
    name_tokenizer="roberta-base",
    path_name_bert_model=roberta_dir,
    path_antonym_pairs=f"{base_path}muse-dlf/data/axis/mft.json",
    dim_names=["virtue", "vice"],
    class_column_names=class_column_names,
    )

In [ ]:
text = "BILL ON IMMIGRANT WORKERS DIES. Legislation to allow nearly twice as many computer-savvy foreigners and other high-skilled immigrants into the country next year apparently has died in Congress. The House passed the compromise measure last month, 288-133, but Sen. Tom Harkin, D-Iowa, had blocked a vote when in the Senate. The proposal, backed by high-tech companies, would raise the limit of so- called H-1B visas granted each year to skilled workers from abroad. Only 65,000 visas are now granted each year; the bill would raise the annual cap to 115,500 for the next two years and to 107,500 in 2001. The ceiling would return to 65,000 in 2002."

In [ ]:
example_dataset, example_dataloader = preprocessor.preprocess_single_article(
    text
)

Output()

RuntimeError: Error loading state dict for SrlBert
	Missing keys: []
	Unexpected keys: ['bert_model.embeddings.position_ids']

## Run model with data

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
Tesla T4


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import torch

def inspect(model, dataloader, K = 14, device='cuda'):
    """
    Make predictions with the given model and dataloader.

    Args:
    - model (torch.nn.Module): The model to make predictions with.
    - dataloader (DataLoader): DataLoader for the dataset to predict on.
    - device (str): Device to make predictions on ('cpu' or 'cuda').

    Returns:
    - predicted_labels (list of lists): List containing the predicted labels for each instance.
    """
    model = model.to("cuda")
    model.eval()

    # dim
    batch_size = dataloader.batch_size
    num_sentences = dataloader.dataset.max_sentences_per_article
    max_args_per_sentence = dataloader.dataset.max_args_per_sentence
    print("num_batches", len(dataloader))
    print("batch_size", batch_size)
    print("num_sentences", num_sentences)
    print("max_args_per_sentence", max_args_per_sentence)
    print("K", K)

    all_preds_span = []

    # Initialize usage lists for each label
    all_used_labels_p = []
    all_used_labels_a0 = []
    all_used_labels_a1 = []

    all_used_fx = []

    with torch.no_grad():
        # Wrap the dataloader with tqdm for batch progress
        for batch in tqdm(dataloader, desc="Processing Batches"):
            sentence_ids = batch['sentence_ids'].to(device)
            sentence_attention_masks = batch['sentence_attention_masks'].to(device)

            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)

            frameaxis_data = batch['frameaxis'].to(device)

            sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = model.aggregation(sentence_ids, sentence_attention_masks, predicate_ids, arg0_ids, arg1_ids)

            # Process each span
            for sentence_idx in range(sentence_embeddings.size(1)):
                s_sentence_span = sentence_embeddings[:, sentence_idx, :]
                v_fx = frameaxis_data[:, sentence_idx, :]

                for span_idx in range(predicate_embeddings.size(2)):
                    v_p_span = predicate_embeddings[:, sentence_idx, span_idx, :]
                    v_a0_span = arg0_embeddings[:, sentence_idx, span_idx, :]
                    v_a1_span = arg1_embeddings[:, sentence_idx, span_idx, :]

                    mask_p = (v_p_span.abs().sum(dim=-1) != 0).float().bool()
                    mask_a0 = (v_a0_span.abs().sum(dim=-1) != 0).float().bool()
                    mask_a1 = (v_a1_span.abs().sum(dim=-1) != 0).float().bool()

                    output = model.unsupervised.combined_autoencoder(
                        v_p_span, v_a0_span, v_a1_span, mask_p, mask_a0, mask_a1, s_sentence_span, 0.6
                    )

                    all_used_labels_p.append(output["p"]["d"].cpu().numpy())
                    all_used_labels_a0.append(output["a0"]["d"].cpu().numpy())
                    all_used_labels_a1.append(output["a1"]["d"].cpu().numpy())

                    del v_p_span, v_a0_span, v_a1_span, mask_p, mask_a0, mask_a1, output
                    torch.cuda.empty_cache()

                mask_fx = (v_fx.abs().sum(dim=-1) != 0).float().bool()

                frameaxis_output = model.unsupervised_fx.frameaxis_autoencoder(v_fx, mask_fx, s_sentence_span, 0.6)

                all_used_fx.append(frameaxis_output["d"].cpu().numpy())

                del v_fx, mask_fx, frameaxis_output
                torch.cuda.empty_cache()

            # Forward pass
            _, span_logits, sentence_logits, combined_logits, _ = model(
                sentence_ids, sentence_attention_masks, predicate_ids, arg0_ids, arg1_ids, frameaxis_data, 0.5
            )
            combined_pred = (torch.sigmoid(combined_logits) > 0.5).float()

            all_preds_span.append(combined_pred.cpu().numpy())

            del sentence_ids, sentence_attention_masks, predicate_ids, arg0_ids, arg1_ids, frameaxis_data
            del sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings
            del span_logits, sentence_logits, combined_logits, combined_pred
            torch.cuda.empty_cache()

    predictions = np.vstack(all_preds_span)

    all_used_labels_p = np.vstack(all_used_labels_p)
    all_used_labels_a0 = np.vstack(all_used_labels_a0)
    all_used_labels_a1 = np.vstack(all_used_labels_a1)

    all_used_fx = np.vstack(all_used_fx)

    # reshape from (iterator (1), num sentences 24, num spans 10, batch size 64, classes 15) to (batch size 64, num sentences 24, num spans 10, classes 15)
    all_used_labels_p = all_used_labels_p.reshape(-1, num_sentences, max_args_per_sentence, K)
    all_used_labels_a0 = all_used_labels_a0.reshape(-1, num_sentences, max_args_per_sentence, K)
    all_used_labels_a1 = all_used_labels_a1.reshape(-1, num_sentences, max_args_per_sentence, K)

    all_used_fx = all_used_fx.reshape(-1, num_sentences, K)

    return predictions, all_used_labels_p, all_used_labels_a0, all_used_labels_a1, all_used_fx


In [ ]:
clean_gpu_memory()

In [ ]:
output = inspect(model, unlabeled_dataloader, device="cuda")

num_batches 7
batch_size 8
num_sentences 64
max_args_per_sentence 20
K 14


Processing Batches:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
predicted_labels, used_labels_p, used_labels_a0, used_labels_a1, used_fx = output

In [ ]:
used_fx[0][0]

array([0.5009594 , 0.49839595, 0.4847231 , 0.5115488 , 0.50410455,
       0.4954454 , 0.50119495, 0.5100542 , 0.5236031 , 0.50039935,
       0.48832914, 0.49564755, 0.4919212 , 0.50229645], dtype=float32)

In [ ]:
# Saving numpy arrays to file
np.savez(base_path + '/labels_data.npz',
         predicted_labels=predicted_labels,
         used_labels_p=used_labels_p,
         used_labels_a0=used_labels_a0,
         used_labels_a1=used_labels_a1,
         used_fx=used_fx)

In [ ]:
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
import pickle

# Initialize category lists
category_lists_p = {category: [] for category in class_column_names}
category_lists_a1 = {category: [] for category in class_column_names}
category_lists_a0 = {category: [] for category in class_column_names}
category_lists_fx = {category: [] for category in class_column_names}

boundary = 0.4

elem_len = len(test_dataloader.dataset)
for elem_idx in tqdm(range(elem_len), desc="Elements"):
    ds = test_dataloader.dataset[elem_idx]

    sent_len = len(ds["predicate_ids"])
    for sentence_idx in tqdm(range(sent_len), desc="Sentences", leave=False):
        span_len = len(ds["predicate_ids"][sentence_idx])
        for span_idx in tqdm(range(span_len), desc="Spans", leave=False):

            # Apply softmax to the current span's labels
            softmaxed_p = F.softmax(torch.tensor(used_labels_p[elem_idx][sentence_idx][span_idx]), dim=-1).numpy()
            softmaxed_a0 = F.softmax(torch.tensor(used_labels_a0[elem_idx][sentence_idx][span_idx]), dim=-1).numpy()
            softmaxed_a1 = F.softmax(torch.tensor(used_labels_a1[elem_idx][sentence_idx][span_idx]), dim=-1).numpy()

            for cat_idx, category in enumerate(class_column_names):
                if softmaxed_p[cat_idx] > boundary:
                    category_lists_p[category].append(ds["predicate_ids"][sentence_idx][span_idx].int().numpy())

                if softmaxed_a0[cat_idx] > boundary:
                    category_lists_a0[category].append(ds["arg0_ids"][sentence_idx][span_idx].int().numpy())

                if softmaxed_a1[cat_idx] > boundary:
                    category_lists_a1[category].append(ds["arg1_ids"][sentence_idx][span_idx].int().numpy())

            # Apply softmax to the current sentence's frame axis labels
            softmaxed_fx = F.softmax(torch.tensor(used_fx[elem_idx][sentence_idx]), dim=-1).numpy()
            if softmaxed_fx[cat_idx] > boundary:
                category_lists_fx[category].append(ds["frameaxis"][sentence_idx].float().numpy())

print("Completed processing all elements.")

IndexError: index 1184 is out of bounds for axis 0 with size 1184

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Ensure you have downloaded the necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def decode_tokens(token_dict, tokenizer, remove_stopwords=False, lemmatize=False):
    decoded_data = {}
    stop_words = set(stopwords.words('english')) if remove_stopwords else set()
    lemmatizer = WordNetLemmatizer() if lemmatize else None

    for category, token_lists in token_dict.items():
        decoded_data[category] = []
        for tokens in token_lists:
            if np.any(tokens > 0):
                # Convert tokens to a list if it's a tensor or numpy array
                if isinstance(tokens, torch.Tensor):
                    tokens = tokens.tolist()
                elif isinstance(tokens, np.ndarray):
                    tokens = tokens.tolist()

                # Decode the tokens
                decoded_text = tokenizer.decode(tokens, skip_special_tokens=True).strip()

                # Remove non-alphabetic characters (but keep spaces)
                decoded_text = re.sub(r'[^A-Za-z ]', '', decoded_text)

                # Tokenize, optionally lemmatize, and remove stop words
                words = word_tokenize(decoded_text)
                processed_words = [lemmatizer.lemmatize(word.lower()) if lemmatizer else word.lower() for word in words if word.lower() not in stop_words]

                # Join the words back into a string and ensure it's not empty
                processed_text = ' '.join(processed_words)
                if processed_text:
                    decoded_data[category].append(processed_text)

    return decoded_data

stop_words = set(stopwords.words('english'))

# Decode the token IDs for each ARG
decoded_predicate = decode_tokens(category_lists_p, tokenizer, remove_stopwords=True, lemmatize=True)
decoded_arg0 = decode_tokens(category_lists_a0, tokenizer, remove_stopwords=True, lemmatize=True)
decoded_arg1 = decode_tokens(category_lists_a1, tokenizer, remove_stopwords=True, lemmatize=True)